In [3]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import altair as alt
from io import BytesIO

In [4]:
df = pd.read_csv('Dados_Tratados.csv')

In [3]:
df.columns

Index(['Gênero', 'Idade', 'Escolaridade', 'Estado Civil', 'Renda',
       'Frequência no Consumo', 'Benefícios da Mentoria',
       'Meta Melhora o Progresso', 'Variedade Melhora a Experiência',
       'Primeira Compra', 'Formato do Infoproduto', 'Área mais Consumida',
       'Satisfação com o Produto', 'Disposição Financeira',
       'Influência na Compra', 'Primeiro Contato', 'Motivação para Compra',
       'Percepção da Experiência com o Produto', 'Indicaria o Produto'],
      dtype='object')

In [65]:
gbar = px.bar(df, x="Área mais Consumida", y="Frequência no Consumo",
              color="Disposição Financeira", barmode="group",
              width=1000, height=500)
gbar = gbar.update_layout(
    title_text='Relação Entre Frequência, Área e Disposição Financeira Para Consumir Infoprodutos',
    title_x=0.5,
    title_font=dict(size=22, family="Arial, sans-serif", color="black"))

In [66]:
pio.write_image(gbar, 'gbar.png')

In [22]:
df_count = df.groupby(['Escolaridade', 'Frequência no Consumo']).size().reset_index(name='count')

# Crie um gráfico de barras usando Altair
chart = alt.Chart(df_count).mark_bar().encode(
    x='Frequência no Consumo:Q',
    y='count:Q',
    color='Escolaridade:N',
    tooltip=['Escolaridade:N', 'Frequência no Consumo:Q', 'count:Q']
)

# Salve o gráfico como um arquivo HTML
chart.save('chart.html')

In [24]:
carr = alt.Chart(df).mark_circle(size=500).encode(
    alt.X('Frequência no Consumo:Q'),
    alt.Y('Disposição Financeira:N'),
    size='count()',
    color='count()',
    tooltip=['count()']
).interactive()

# Exiba o gráfico
carr.save('carr.html')

In [59]:
df_agrupado = df.groupby('Influência na Compra')['Frequência no Consumo'].sum().reset_index()

# Calcule as porcentagens
df_agrupado['Porcentagem'] = df_agrupado['Frequência no Consumo'] / df_agrupado['Frequência no Consumo'].sum() * 100

# Crie um gráfico de pizza/setores usando Altair
cizza = alt.Chart(df_agrupado).mark_arc().encode(
    alt.Color('Influência na Compra:N', scale=alt.Scale(scheme='category10')),
    tooltip=['Influência na Compra:N', 'Frequência no Consumo:Q', 'Porcentagem:Q'],
    size='Frequência no Consumo:Q',
    theta='Porcentagem:Q'
).properties(
    width=300,
    height=300
).interactive()

# Adicione rótulos de porcentagem nas fatias
labels = alt.Chart(df_agrupado).mark_text(align='center', baseline='middle', fontSize=12).encode(
    text='Porcentagem:N',
    color=alt.value('black'),
    radius=alt.value(150),  # Ajuste a distância dos rótulos do centro
    theta='theta:O'  # Use 'theta:O' em vez de 'theta:N'
).transform_calculate(
    Porcentagem='datum.Porcentagem',
    theta='360 * datum.Porcentagem / 100'  # Ajuste o cálculo do ângulo
)

# Exiba o gráfico com rótulos de porcentagem
chart_final = (cizza + labels).configure_view(stroke=None)
chart_final.save('cizza.html')